In [0]:
import numpy as np
import pandas as pd
import nltk
import spacy
import os
import string
import gensim

In [0]:
train = pd.read_csv("/content/train_2.csv")
train = train.dropna()
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,Q1,Q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"['step', 'step', 'guide', 'invest', 'share', '...","['step', 'step', 'guide', 'invest', 'share', '..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"['story', 'kohinoor', 'kohinoor', 'diamond']","['would', 'happen', 'indian', 'government', 's..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"['increase', 'speed', 'internet', 'connection'...","['internet', 'speed', 'increased', 'hacking', ..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"['mentally', 'lonely', 'solve']","['find', 'remainder', 'math2324math', 'divided..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"['one', 'dissolve', 'water', 'quikly', 'sugar'...","['fish', 'would', 'survive', 'salt', 'water']"


In [0]:
!wget -c http://nlp.stanford.edu/data/glove.840B.300d.zip
!wget -c https://dl.fbaipublicfiles.com/infersent/infersent1.pkl

In [0]:
!wget -c https://raw.githubusercontent.com/facebookresearch/InferSent/master/models.py

In [0]:
import zipfile
with zipfile.ZipFile("/content/glove.840B.300d.zip", "r") as zipread:
  zipread.extractall("/content/")
  zipread.close

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import torch
from models import InferSent

MODEL_PATH = '/content/infersent1.pkl'
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 2}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

infersent.set_w2v_path("/content/glove.840B.300d.txt")

In [0]:
import itertools
from itertools import chain

print(len(train))
infersent.build_vocab(list(chain(train.question1, train.question2)), tokenize=True)

404287
Found 102076(/137513) words with w2v vectors
Vocab size : 102076


In [0]:
embeddings1 = infersent.encode(train.question1, tokenize=True)

In [0]:
embeddings2 = infersent.encode(train.question2, tokenize=True)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
infersent_cos_sim = []
for (e1, e2) in zip(embeddings1, embeddings2):
  temp = cosine_similarity(e1.reshape(1,-1), e2.reshape(1,-1))[0][0]
  infersent_cos_sim.append(temp)
train["infersent_cos_sim"] = infersent_cos_sim
train.head()

In [0]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train[["cos_sim"]], train["is_duplicate"], random_state=5, test_size=0.2)

In [0]:
import xgboost as xgb
options = {"model_param":
         {"eta": 0.15, 
         "max_depth": 4,
         "n_estimators":50,
         "sub_sample":0.5,
         "scale_pos_weight":1.75,
         "random_state":1,
         "eval_metric": "logloss",
         "objective": "binary:logistic"},
         "num_round":50,
         "test_frac":0.2}

In [0]:
#training model
xgbmatrix = xgb.DMatrix(train_X, train_y)
xgb_model = xgb.train(options["model_param"], xgbmatrix, options["num_round"], verbose_eval=10)
xgb_valmatrix = xgb.DMatrix(val_X)
y_val_pred = xgb_model.predict(xgb_valmatrix)
y_val_pred #just checking

In [0]:
y_val_pred_binary = [1 if y>0.5 else 0 for y in y_val_pred]
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(val_y, y_val_pred_binary)

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, y_val_pred_binary)